# Test macro for Bokeh client visualization
* it is aimed to be used for develoopers to test feature of bokeh interactive  - standalone plotting

In [ ]:
from bokeh.io import output_notebook, show
#, output_file
from bokeh.plotting import figure
from bokeh.layouts import column, row
from bokeh.models import CustomJS, ColumnDataSource, Slider
from RootInteractive.InteractiveDrawing.bokeh.bokehTools import *
from RootInteractive.InteractiveDrawing.bokeh.bokehDraw import *
output_notebook()


In [ ]:
#init test random data
df = pd.DataFrame(np.random.randint(0, 100, size=(100, 4)), columns=list('ABCD'))
df["Category"]=df.eval("A%5")
df.head(10)
df.metaData = {'A.AxisTitle': "A (cm)", 'B.AxisTitle': "B (cm/s)", 'C.AxisTitle': "C (s)", 'D.AxisTitle': "D (a.u.)"}
#print(df.metaData)

In [ ]:
bokehDrawArray?

In [ ]:
figureArray = [
        [['A'], ['D+A','C-A'], {"size": 1}],
        [['A'], ['C+A', 'C-A']],
        [['A'], ['Category']],
    ]
figureLayout: str = '((0,1),(2, x_visible=1),commonX=1,x_visible=1,y_visible=0,plot_height=250,plot_width=1000)'
tooltips = [("VarA", "(@A)"), ("VarB", "(@B)"), ("VarC", "(@C)"), ("VarD", "(@D)")]
pAll,layoutList,source =bokehDrawArray(df, "A>0", figureArray, layout=figureLayout, color="blue", size=4, tooltips=tooltips)
show(pAll)

In [ ]:
logging.disable(1)

In [ ]:
output_file("test2.html")
def makeJScallback(widgetDict):
    size=widgetDict['cdsOrig'].data["index"].size
    code=\
    """
    var dataOrig = cdsOrig.data;
    var dataSel = cdsSel.data;
    console.log('%f\t%f\t',dataOrig["A"].length, dataSel["A"].length);
    """
    for a in widgetDict['cdsOrig'].data:
        code+=f"dataSel[\'{a}\']=[];\n"
    code+=f"""var arraySize={size};\n"""
    code+="""var nSelected=0;\n"""
    code+=f"""for (var i = 0; i < {size}; i++)\n"""
    code+=" {\n"
    code+="""var isSelected=1;\n"""
    for key, value in widgetDict.items():
        if type(value).__name__ == "RangeSlider":
            dataName=key.replace("Range","")
            code+=f"      var {key}Value={key}.value;\n"
            code+=f"      console.log(\"%s\t%f\t%f\t%f\",\"{key}\",{key}Value[0],{key}Value[1],dataOrig[\"{dataName}\"][i]);\n"
            code+=f"      isSelected&=(dataOrig[\"{dataName}\"][i]>={key}Value[0])\n"
            code+=f"      isSelected&=(dataOrig[\"{dataName}\"][i]<={key}Value[1])\n"
            #print(value)
    code+="""      
        console.log(\"isSelected:%d\t%d\",i,isSelected);
        if (isSelected) nSelected++;
        if (isSelected){
    """
    for a in widgetDict['cdsOrig'].data:
        code+=f"dataSel[\'{a}\'].push(dataOrig[\'{a}\'][i]);\n"
    code+="""
        }
    }
    console.log(\"nSelected:%d\",nSelected); 
    cdsSel.change.emit();
    """
    print(code)
    callback = CustomJS(args=widgetDict, code=code)
    #print(callback)
    #display(widgetDict)
    return callback

def test_FigureArray():
    """
    Test of the bokehDrawArray to draw figure array -
    :return:
    """
    # 1.) create figures
    figureArray = [
        [['A'], ['D+A','C-A'], {"size": 8}],
        [['A'], ['C+A', 'C-A']],
        [['A'], ['sin(A/10)', 'sin(A/20)*0.5', 'sin(A/40)*0.25'], {"size": 10}]
    ]
    figureLayout: str = '((0,1, plot_height=200),(2, x_visible=1),commonX=1,x_visible=1,y_visible=0,plot_height=150,plot_width=1000)'
    tooltips = [("VarA", "(@A)"), ("VarB", "(@B)"), ("VarC", "(@C)"), ("VarD", "(@D)")]
    #dummy,source,figureList  = bokehDrawArray(df, "A>0", figureArray, layout=figureLayout, color="blue", size=4, tooltips=tooltips)
    dummy,source,figureList  = bokehDrawArray(df, "A>0", figureArray, layout=figureLayout, color="blue", size=4, tooltips=tooltips)

    # 2.) make CDS
    cdsOrig = ColumnDataSource(df.query("A>0"))
    cdsSel =  source
    widgetDict={"cdsOrig":cdsOrig, "cdsSel":cdsSel}
    sliders=[]
    # 3.) create sliders 
    for a in df.columns:
        length=df[a].max()-df[a].min()
        sliderRange= RangeSlider(start=df[a].min(), end=df[a].max(), value=(df[a].min(), df[a].max()), step=length*0.01, title=a+"Range")
        sliders.append([sliderRange])
        widgetDict[a+"Range"]=sliderRange
    # make callback
    mycallback=makeJScallback(widgetDict)
    display(mycallback)
    for a in sliders:
        figureList.append(a)
        a[0].js_on_event('value',mycallback)
        a[0].js_on_change('value',mycallback)
        
    sliderRange2= RangeSlider(start=0.1, end=4, value=(0.1,4) , step=0.1, title="Range2")
    sliderRange2.js_on_change('value', mycallback)
    figureList.append([sliderRange2])
    pAll=    gridplotRow(figureList)
    show(pAll)
    return pAll
    #print(widgetDict)
    
figure=test_FigureArray() 